<a href="https://colab.research.google.com/github/BeatrixBlaine/DS-C1/blob/main/3_5_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Structure

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [1]:
class ReasoningState:
    def __init__(self):
        self.steps = []
        self.query_history = []
        self.confidence_history = []

    def update(self, reasoning, confidence):
        self.steps.append(reasoning)
        self.confidence_history.append(confidence)


In [3]:
# Confidence Estimation

def reasoning_confidence(logits):
    """
    Confidence proxy from model output logits or probabilities
    """
    probs = np.exp(logits) / np.sum(np.exp(logits))
    entropy = -np.sum(probs * np.log(probs + 1e-12))
    return 1.0 - entropy / np.log(len(probs))


In [4]:
# Retrieval Function

def retrieve(query_embedding, corpus_embeddings, corpus, top_k=3):
    sims = cosine_similarity(
        query_embedding.reshape(1, -1),
        corpus_embeddings
    ).flatten()

    top_idx = np.argsort(sims)[-top_k:][::-1]
    return [corpus[i] for i in top_idx], sims[top_idx]


In [6]:
# Reasoning Step

def reasoning_step(context, retrieved_docs):
    """
    Simulated reasoning function
    """
    combined = context + " " + " ".join(retrieved_docs)

    # Fake logits (stand-in for model output)
    logits = np.random.randn(5)

    reasoning_output = f"Reasoning based on: {combined[:100]}..."
    confidence = reasoning_confidence(logits)

    return reasoning_output, confidence


In [7]:
# Iterative Reasoning-Retrieval

def iterative_reasoning_retrieval(
    query,
    query_embedding,
    corpus,
    corpus_embeddings,
    max_steps=5,
    confidence_threshold=0.85
):
    state = ReasoningState()
    context = query

    for step in range(max_steps):
        # Retrieve
        docs, scores = retrieve(
            query_embedding,
            corpus_embeddings,
            corpus
        )

        # Reason
        reasoning, conf = reasoning_step(context, docs)
        state.update(reasoning, conf)

        # Update context
        context += " " + reasoning

        # Adaptive stopping
        if conf >= confidence_threshold:
            break

    return state


In [8]:
# Example Usage

# Dummy corpus
corpus = [
    "In-context learning improves with relevant examples.",
    "Retrieval augmented generation combines search and reasoning.",
    "Entropy reflects uncertainty in predictions.",
    "Diversity helps coverage of the input space."
]

corpus_embeddings = np.random.randn(len(corpus), 128)
query_embedding = np.random.randn(128)

state = iterative_reasoning_retrieval(
    query="How does retrieval help reasoning?",
    query_embedding=query_embedding,
    corpus=corpus,
    corpus_embeddings=corpus_embeddings
)

for i, step in enumerate(state.steps):
    print(f"Step {i+1}: {step}")


Step 1: Reasoning based on: How does retrieval help reasoning? Retrieval augmented generation combines search and reasoning. In-...
Step 2: Reasoning based on: How does retrieval help reasoning? Reasoning based on: How does retrieval help reasoning? Retrieval ...
Step 3: Reasoning based on: How does retrieval help reasoning? Reasoning based on: How does retrieval help reasoning? Retrieval ...
Step 4: Reasoning based on: How does retrieval help reasoning? Reasoning based on: How does retrieval help reasoning? Retrieval ...
Step 5: Reasoning based on: How does retrieval help reasoning? Reasoning based on: How does retrieval help reasoning? Retrieval ...
